**IMPORT NECESSARY LIBRARIES**

In [2]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

**MOUNT GOOGLE DRIVE**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**READ THE DATASET**

In [4]:
text_file = "/content/drive/MyDrive/Colab Notebooks/Mini_project_3/2dataset.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break

Go.	යන්න.
Hi.	ආයුබෝවන්.
Run.	දුවන්න.
Who?	WHO?
Wow!	වාව්!
Fire!	ගිනි!
Help!	උදව්!
Jump!	පනින්න!
Jump.	පනින්න.
Stop!	නවත්වන්න!
Wait!	ඉන්න!
Go on.	යන්න.
Hello!	ආයුබෝවන්!
Hurry!	ඉක්මන් කරන්න!
I see.	මම දකියි.
I try.	මම උත්සාහ කරනවා.
I won!	මම දිනුවා!
Oh no!	අපොයි නෑ!
Relax.	සන්සුන් වන්න.
Shoot!	වෙඩි තියන්න!


In [5]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

I make it a practice to help my brother with his homework after supper.	රාත්‍රී ආහාරයෙන් පසු මගේ සහෝදරයාගේ ගෙදර වැඩවලට උදව් කිරීම මම පුරුද්දක් කරගනිමි.
Instead of laying off these workers, why don't we just cut their hours?	මේ කම්කරුවන් දොට්ට දමනවා වෙනුවට අපි ඔවුන්ගේ පැය ගණන කපා නොගන්නේ මන්ද?
The thieves pulled open all the drawers of the desk in search of money.	හොරු සල්ලි හොයන්න මේසයේ ලාච්චු සේරම ඇරියා.
Father kept in touch with us by mail and telephone while he was overseas.	තාත්තා විදේශගතව සිටියදී තැපෑලෙන් සහ දුරකථනයෙන් අපිව සම්බන්ධ කරගත්තා.
George Washington was the first president of the Unites States of America.	ජෝර්ජ් වොෂින්ටන් ඇමරිකා එක්සත් ජනපදයේ පළමු ජනාධිපති විය.
Mother Teresa used the prize money for her work in India and around the world.	තෙරේසා මවුතුමිය එම ත්‍යාග මුදල ඉන්දියාවේ සහ ලොව පුරා සිය කටයුතු සඳහා යෙදවූවාය.
If you go to that supermarket, you can buy most things you use in your daily life.	ඔබ එම සුපිරි වෙළඳසැලට ගියහොත්, ඔබ එදිනෙදා ජීවිතයේ භාවිතා කරන බොහෝ දේ ඔබට මි

**SPLIT THE ENGLISH AND SINHALA TRANSLATION PAIRS**

In [6]:
text_pairs = []
for line in lines:
    english, sinhala = line.split("\t")
    sinhala = "[start] " + sinhala + " [end]"
    text_pairs.append((english, sinhala))
for i in range(3):
  print(random.choice(text_pairs))

('This machine was manufactured in France.', '[start] මෙම යන්ත්රය ප්රංශයේ නිෂ්පාදනය කරන ලදී. [end]')
('I enjoyed swimming in the river.', '[start] මම ගඟේ පිහිනීමට ප්\u200dරිය කළෙමි. [end]')
("I couldn't stop myself from longing for her.", '[start] මට ඇය ගැන ආසාවෙන් ඉන්න එක නවත්වන්න බැරි උනා. [end]')


**RANDOMIZE THE DATA**

In [7]:
import random
random.shuffle(text_pairs)

**SPITING THE DATASET INTO TRAINING, TESTING AND VALIDATION DATA**

In [8]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))
print("Total size of the dataset:",len(train_pairs)+len(val_pairs)+len(test_pairs))


Total sentences: 49201
Training set size: 34441
Validation set size: 7380
Testing set size: 7380
Total size of the dataset: 49201


**REMOVING PUNCTUATIONS**

In [9]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [10]:
f"{3+5}"

'8'

**VECTORIZATION THE ENGLISH AND SINHALA TEXT PAIRS**
---



In [11]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]


In [12]:
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

**PREPARATION OF THE DATASET FRO THE TRANSLATION**

In [13]:
batch_size = 64
def format_dataset(eng, sin):
    eng = source_vectorization(eng)
    sin = target_vectorization(sin)
    return ({
        "english": eng,
        "sinhala": sin[:, :-1],
    }, sin[:, 1:])
def make_dataset(pairs):
    eng_texts, sin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    sin_texts = list(sin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")
inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[50])

inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
({'english': array([[  50,  124,  989, ...,    0,    0,    0],
       [   9,  347,  121, ...,    0,    0,    0],
       [  20,   13, 2420, ...,    0,    0,    0],
       ...,
       [  91,   11,    2, ...,    0,    0,    0],
       [  38,    8,    6, ...,    0,    0,    0],
       [  44,  782,    4, ...,    0,    0,    0]]), 'sinhala': array([[   2,   12, 1248, ...,    0,    0,    0],
       [   2,    7, 3340, ...,    0,    0,    0],
       [   2,   10,   54, ...,    0,    0,    0],
       ...,
       [   2,  290,  484, ...,    0,    0,    0],
       [   2,   31, 2351, ...,    0,    0,    0],
       [   2, 5607,  589, ...,    0,    0,    0]])}, array([[  12, 1248,  406, ...,    0,    0,    0],
       [   7, 3340,  713, ...,    0,    0,    0],
       [  10,   54,   45, ...,    0,    0,    0],
       ...,
       [ 290,  484,  181, ...,    0,    0,    0],
       [  31, 2351, 4692, ...,    0,    0, 

**TRANSFORMER ENCODER IMPLEMENTED AS A SUBCLASSED LAYER**

In [14]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**THE TRANSFORMER DECODER**

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


**POSITIONAL ENCODEING**

In [16]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**END-TO-END TRANSFORMER**

In [17]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

**TRAINING THE TRANSFORMER NEURAL NETWORK**

In [19]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=40, validation_data=val_ds)

Epoch 1/40
539/539 [==============================] - 52s 78ms/step - loss: 5.1399 - accuracy: 0.3492 - val_loss: 4.2036 - val_accuracy: 0.4123
Epoch 2/40
539/539 [==============================] - 35s 65ms/step - loss: 4.0324 - accuracy: 0.4414 - val_loss: 3.6107 - val_accuracy: 0.4776
Epoch 3/40
539/539 [==============================] - 36s 67ms/step - loss: 3.5051 - accuracy: 0.4929 - val_loss: 3.3434 - val_accuracy: 0.5081
Epoch 4/40
539/539 [==============================] - 37s 68ms/step - loss: 3.1652 - accuracy: 0.5295 - val_loss: 3.2115 - val_accuracy: 0.5239
Epoch 5/40
539/539 [==============================] - 36s 68ms/step - loss: 2.9294 - accuracy: 0.5571 - val_loss: 3.1047 - val_accuracy: 0.5422
Epoch 6/40
539/539 [==============================] - 37s 68ms/step - loss: 2.7464 - accuracy: 0.5801 - val_loss: 3.0772 - val_accuracy: 0.5441
Epoch 7/40
539/539 [==============================] - 37s 68ms/step - loss: 2.6176 - accuracy: 0.5993 - val_loss: 3.0618 - val_accuracy:

**MANUAL TESTING OF THE TRANSLATION MODEL WITH 20 NEW SENTENCES**

In [20]:
import numpy as np
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
There's not enough room in here for both Tom and Mary.
[start] ටොම් දෙදෙනාම සිතන්නේ එම දොර ගැන එපා දෙදෙනාම චිත්‍ර සහ දුරකථන අංකය නොවේ [end]
-
Is it normal?
[start] එය සාමාන්‍ය දෙයක් ද [end]
-
That's the most important thing.
[start] එය වැදගත් වන්නේ හොඳම ය [end]
-
I had to walk home.
[start] මට ගෙදර යන්න වුණා [end]
-
There is no hope that he will come soon.
[start] ඔහු ඉක්මනින්ම හෙට ළඟදීම එනවා [end]
-
I will leave it to your judgement.
[start] මම එය ඔබගේ [UNK] පිටත් වෙමි [end]
-
The engine doesn't work.
[start] පසුගිය පසු වැඩ කරන්නේ නැහැ [end]
-
I got some shampoo in my eyes and it stings.
[start] මගේ හිසකෙස් සෝදා ගැනීම මගේ ඇස් වල දුම් පානය කළා [end]
-
Since he was very drunk, he couldn't drive his car home.
[start] ඔහුට මෝටර් රථයක් පැමිණ සිටි පරිදි ඔහුගේ මෝටර් රථය දැඩි ලෙස කෑම වලට නොහැකි විය [end]
-
Have fun.
[start] විහිළු කරන්න [end]
-
She makes a point of taking a shower before breakfast.
[start] ඇය උදේ කෑම කන ගමන් කරයි [end]
-
We go there often.
[start] අපි බොහෝ විට කෑම යනවා [end